In [1]:
#Import PyTorch
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torch import optim

#Import TorchVision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
import torchvision.models as models
from torchvision.models import resnet
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

#torchinfo for printing a model summary
from torchinfo import summary

#import other libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import seaborn as sns
import os
import random 
import shutil
from tqdm.auto import tqdm
import zipfile
import pickle
import joblib

In [2]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
def class_labels_reassign(age):
    if 1 <= age <= 10:
        return 0
    elif 11 <= age <= 20:
        return 1
    elif 21 <= age <= 30:
        return 2
    elif 31 <= age <= 40:
        return 3
    elif 41 <= age <= 50:
        return 4
    elif 51 <= age <= 60:
        return 5
    elif 61 <= age <= 70:
        return 6
    elif 71 <= age <= 80:
        return 7
    elif 81 <= age <= 90:
        return 8
    else:
        return 9

class CustomDataset(Dataset):
    def __init__(self, data_dir,transform=None):
        self.data_dir = data_dir
        self.img_paths = sorted(os.listdir(data_dir))
        self.transforms = transforms.Compose([
            transforms.Resize((224, 112)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])
        ])    
    def __len__(self):
        return len(self.img_paths)    
    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.img_paths[idx])
        img = Image.open(img_path).convert('RGB')
        age = int(img_path.split('/')[-1].split('_')[0])
        age_label = class_labels_reassign(age) 
        gender_label = int(img_path.split('/')[-1].split('_')[1])
        img = self.transforms(img)
        return img, age_label , gender_label
    
    
train_data = CustomDataset('/kaggle/input/utk-cropped-img/aUTK_cropped/utk_cropped_train')
train_loader = DataLoader(train_data, batch_size=64, shuffle=True,drop_last=True)

test_data = CustomDataset('/kaggle/input/utk-cropped-img/aUTK_cropped/utk_cropped_test')
test_loader = DataLoader(test_data, batch_size=64, shuffle=False,drop_last=True)



In [4]:
# img_path = '/kaggle/input/utk-cropped-img/aUTK_cropped/utk_cropped_test/100_1_0_20170110183726390.jpg.chip_right.jpg'
# img = Image.open(img_path).convert('RGB')
# age_label = int(img_path.split('/')[-1].split('_')[0])
# age_interval = class_labels_reassign(age_label)
# gender_label = int(img_path.split('/')[-1].split('_')[1])
# print(age_interval,gender_label)


In [5]:
### The Residual block consists of :
#  Conv_layer_1 ---> Batch_Normalization_1 ---> ReLU_activation_function_1 
#  ---> Conv_layer_2 ---> Batch_Normalization_2 ---> ReLU_activation_function_2 

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
                
        return out


#  The skip connection function consists of simply 2 layers :
#  Conv_layer ---> MaxPool_layer
 
class SkipConnection1 (nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SkipConnection1, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels=3, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=8, stride=8, padding=1)
        
    def forward(self, x):
        
        out = self.conv(x)
        out = self.pool(out)
        
        return out
    
class SkipConnection2 (nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SkipConnection2, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels=3, kernel_size=1, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=5, stride=4, padding=0)
        
    def forward(self, x):
        out = self.conv(x)
        out = self.pool(out)
        return out

class SkipConnection3(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SkipConnection3, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels=3, kernel_size=1, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=4, stride=2, padding=0)

    def forward(self, x):
        out = self.conv(x)
        out = self.pool(out)
        return out


class SkipConnection4(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SkipConnection4, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels=3, kernel_size=1, stride=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=(1,2), stride=(1,1))
        
    def forward(self, x):
        out = self.conv(x)
        out = self.pool(out)
        return out


    
#   The Perigender Layer is a combination of the aforementioned function in the following fashion : 
#    
#      1. Conv_layer_1 --> MaxPool2d_1 
#      2. Skip_function_1
#      3. Residual_batch_1             
#      4. Skip_function_2
#      5. Residual_batch_2
#      6. Skip_function_3
#      7. Residual_function_3
#      8. Skip_function_4
#      9. Residual_function_4  
#     10. MaxPool2d_2
#     11. Concatenation_layer
#     12. AvgPool_layer


class PeriOcular(nn.Module):
    def __init__(self, num_classes=2):
        super(PeriOcular, self).__init__()

        self.conv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2,padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2,padding=1)
        
        self.skip1 = SkipConnection1(64, 64)
        self.resblock1 = nn.Sequential(ResBlock(64, 64, stride=2), ResBlock(64, 128))
        
        self.skip2 = SkipConnection2(128, 128)
        self.resblock2 = nn.Sequential(ResBlock(128, 128, stride=2), ResBlock(128, 256))
        
        self.skip3 = SkipConnection3(256, 256)
        self.resblock3 = nn.Sequential(ResBlock(256, 256, stride=2), ResBlock(256, 512))
        
        self.skip4 = SkipConnection4(512, 4)
        self.resblock4 = nn.Sequential(ResBlock(512, 512, stride=2), ResBlock(512, 512))
        
        self.maxpool2 = nn.MaxPool2d(kernel_size=(1,2), stride=(1,1))
        
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.5)
        
        self.fc = nn.Linear(524, num_classes)
        self.fc2 = nn.Linear(524,10)
        

    def forward(self, x):
                   
        out = self.conv(x)        
        out = self.maxpool1(out)
        
        skip1 = self.skip1(out)
        out = self.resblock1(out)
                
        skip2 = self.skip2(out)
        out = self.resblock2(out)
                
        skip3 = self.skip3(out)
        out = self.resblock3(out)
                
        skip4 = self.skip4(out)
                
        out = self.maxpool2(out)
                
        skip_outputs = [skip1, skip2, skip3, skip4,out]
        concatenated = torch.cat(skip_outputs, dim=1)
        
        out = self.avg_pool(concatenated)
        out = out.squeeze()
        out = self.dropout(out)
        out1 = out        
        out = self.fc(out)
        out1 = self.fc2(out1)
               
        return out,out1


In [6]:
# model = PeriOcular()
# batch_size = 64
# summary(model, input_size=(batch_size, 3, 224, 112))

In [7]:
def train(model, train_loader, optimizer, loss_fn, device):
    
    #set the model to training mode
    model.train()
    train_loss = 0
    train_acc = 0
    
    for images, age_labels, gender_labels in train_loader:
        
        #put the images and labels on the device
        images = images.to(device)
        age_labels = age_labels.to(device)
        gender_labels = gender_labels.to(device)
        
        #optimizer zero grad
        optimizer.zero_grad()
        
        #send the images to the model and get the outputs
        gender_outputs, age_outputs = model(images)
        
        #calculate the loss
        age_loss = loss_fn(age_outputs, age_labels)
        gender_loss = loss_fn(gender_outputs, gender_labels)
        loss = age_loss + gender_loss
        
        #loss backward
        loss.backward()
        
        #optmizer step
        optimizer.step()
        
        #total the loss and accuracy so we can print it
        train_loss += loss.item() * images.size(0)
        _, predicted_age = torch.max(age_outputs, 1)
        _, predicted_gender = torch.max(gender_outputs, 1)
        train_acc += (predicted_age.eq(age_labels.data) & predicted_gender.eq(gender_labels.data)).sum().item()
    
    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)
    
    return train_loss, train_acc

def test(model, test_loader, loss_fn, device):
    
    #set the model to eval mode for testing
    model.eval()
    test_loss = 0
    test_acc = 0
    
    with torch.no_grad():
        for images, age_labels, gender_labels in test_loader:
            
            #put the labels and images on the correct device
            images = images.to(device)
            age_labels = age_labels.to(device)
            gender_labels = gender_labels.to(device)
            
            #send the images to the model and get the outputs
            gender_outputs, age_outputs = model(images)      
            
            #calculate the loss
            age_loss = loss_fn(age_outputs, age_labels)
            gender_loss = loss_fn(gender_outputs, gender_labels)
            
            #find the total loss and accuracy
            loss = age_loss + gender_loss
            test_loss += loss.item() * images.size(0)
            _, predicted_age = torch.max(age_outputs, 1)
            _, predicted_gender = torch.max(gender_outputs, 1)
            test_acc += (predicted_age.eq(age_labels.data) & predicted_gender.eq(gender_labels.data)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_acc /= len(test_loader.dataset)

    return test_loss, test_acc

model = PeriOcular()
model.to(device)
num_epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in tqdm(range(num_epochs)):
    train_loss, train_acc = train(model, train_loader, optimizer, loss_fn, device)
    test_loss, test_acc = test(model, test_loader, loss_fn, device)

    print(f'Epoch: {epoch+1}')
    print(f'Train Loss: {train_loss:.4f}', f'Train Accuracy: {train_acc:.4f}')
    print(f'Test Loss: {test_loss:.4f}', f'Test Accuracy: {test_acc:.4f}')



  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1
Train Loss: 2.6441 Train Accuracy: 0.1702
Test Loss: 2.4866 Test Accuracy: 0.1922
Epoch: 2
Train Loss: 2.4389 Train Accuracy: 0.2195
Test Loss: 2.3408 Test Accuracy: 0.2467
Epoch: 3
Train Loss: 2.3168 Train Accuracy: 0.2611
Test Loss: 2.2493 Test Accuracy: 0.2632
Epoch: 4
Train Loss: 2.2599 Train Accuracy: 0.2750
Test Loss: 2.2702 Test Accuracy: 0.2635
Epoch: 5
Train Loss: 2.2157 Train Accuracy: 0.2814
Test Loss: 2.1624 Test Accuracy: 0.2944
Epoch: 6
Train Loss: 2.1923 Train Accuracy: 0.2859
Test Loss: 2.1732 Test Accuracy: 0.2796
Epoch: 7
Train Loss: 2.1606 Train Accuracy: 0.2933
Test Loss: 2.1258 Test Accuracy: 0.2999
Epoch: 8
Train Loss: 2.1345 Train Accuracy: 0.2993
Test Loss: 2.1139 Test Accuracy: 0.2966
Epoch: 9
Train Loss: 2.1022 Train Accuracy: 0.3045
Test Loss: 2.0653 Test Accuracy: 0.3029
Epoch: 10
Train Loss: 2.0870 Train Accuracy: 0.3035
Test Loss: 2.0618 Test Accuracy: 0.3015
Epoch: 11
Train Loss: 2.0661 Train Accuracy: 0.3101
Test Loss: 2.0522 Test Accuracy: 0.31

In [8]:
torch.save(model.state_dict(),'PeriOcular_v2.pth')

In [9]:
model.load_state_dict(torch.load('/kaggle/input/combined-model/PeriOcular_v1.pth'))

<All keys matched successfully>

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model

# define classes for gender
gender_classes = ['female', 'male']

# define function to get age range
def get_age_range(age):
    if age == 0:
        return "Age is in range 1-10"
    elif age == 1: 
        return "Age is in range 11-20"
    elif age == 2:
        return "Age is in range 21-30"
    elif age == 3:
        return "Age is in range 31-40"
    elif age == 4:
        return "Age is in range 41-50"
    elif age == 5:
        return "Age is in range 51-60"
    elif age == 6:
        return "Age is in range 61-70"
    elif age == 7:
        return "Age is in range 71-80"
    elif age == 8:
        return "Age is in range 81-90"
    else:
        return "Age is above 90"

# define function to preprocess image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 112)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)
    return image

# define function to predict gender and age for an image
def predict(model, image_path):
    image = preprocess_image(image_path)
    image = image.to(device)
    model.eval()
    with torch.no_grad():
        gender_output, age_output = model(image)
        gender_index = torch.argmax(gender_output).item()
        age_index = torch.argmax(age_output).item()
    gender = gender_classes[gender_index]
    age_range = get_age_range(age_index)
    return gender, age_range

# test a random image
image_path = '/kaggle/input/utk-cropped-img/aUTK_cropped/utk_cropped_test/10_0_0_20170110220557169.jpg.chip_right.jpg'
predicted_gender, predicted_age_range = predict(model, image_path)
print('Predicted Gender:', predicted_gender)
print('Predicted Age Range:', predicted_age_range)


Predicted Gender: male
Predicted Age Range: Age is in range 21-30
